In [12]:
# 🔹 1. นำเข้าไลบรารี
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments
import os

In [2]:
# ตรวจสอบ CPU, GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 🔹 3. โหลดชุดข้อมูล (Dataset)
dataset = load_dataset("Thaweewat/alpaca-cleaned-52k-th")

In [4]:
# ดูตัวอย่างข้อมูล
print(dataset["train"][0])  # ตัวอย่างข้อมูลจากชุดข้อมูล train

{'instruction': ' ให้สามเคล็ดลับในการรักษาสุขภาพ', 'input': '', 'output': ' 1. รับประทานอาหารที่สมดุลและมีคุณค่าทางโภชนาการ: ตรวจสอบให้แน่ใจว่ามื้ออาหารของคุณประกอบด้วยผักและผลไม้หลากหลายชนิด โปรตีนไม่ติดมัน เมล็ดธัญพืช และไขมันที่ดีต่อสุขภาพ สิ่งนี้ช่วยให้ร่างกายของคุณได้รับสารอาหารที่จำเป็นเพื่อให้ทำงานได้ดีที่สุดและสามารถช่วยป้องกันโรคเรื้อรังได้ 2. ออกกำลังกายอย่างสม่ำเสมอ: การออกกำลังกายเป็นสิ่งสำคัญสำหรับการรักษากระดูก กล้ามเนื้อ และสุขภาพหัวใจและหลอดเลือดให้แข็งแรง ตั้งเป้าหมายให้ออกกำลังกายแบบแอโรบิกระดับปานกลางอย่างน้อย 150 นาทีหรือออกกำลังกายแบบหนัก 75 นาทีต่อสัปดาห์ 3. นอนหลับให้เพียงพอ: การนอนหลับที่มีคุณภาพเพียงพอเป็นสิ่งสำคัญสำหรับความเป็นอยู่ที่ดีทั้งทางร่างกายและจิตใจ ช่วยควบคุมอารมณ์ ปรับปรุงการทำงานของสมอง และสนับสนุนการเจริญเติบโตอย่างแข็งแรงและการทำงานของภูมิคุ้มกัน ตั้งเป้าหมายการนอนหลับให้ได้ 7-9 ชั่วโมงในแต่ละคืน'}


In [5]:
# แบ่งข้อมูล train เป็น train และ validation (80% train, 20% validation)
train_dataset = dataset["train"].train_test_split(test_size=0.2)
val_dataset = train_dataset["test"]
train_dataset = train_dataset["train"]

In [7]:
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 41408
})
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10352
})


In [109]:
# 🔹 4. โหลด Base Model และ Tokenizer (ใช้ GPU)
model_name = "Pavarissy/phayathaibert-thainer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

If you want to use `CamembertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of CamembertForCausalLM were not initialized from the model checkpoint at Pavarissy/phayathaibert-thainer and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ย้ายโมเดลไปที่ GPU
model.to(device)

CamembertForCausalLM(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(249262, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

In [66]:
# ทดสอบ model
input_text = "กินข้าวดีไหม"

# ทำการ tokenize ข้อความ input
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

# ใช้โมเดลในการสร้าง output
with torch.no_grad():  # ไม่ต้องคำนวณ gradient
    output = model.generate(inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)

# แปลง output ที่เป็น token กลับเป็นข้อความ
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# แสดงผลลัพธ์
print(f"Input: {input_text}")
print(f"Output: {output_text}")

Input: กินข้าวดีไหม
Output: กินข้าวดีไหมถึงบ้านจะกลับประพฤติเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการเปิดทําการตั้งพรรคตั้งพรรคโหมดิตี้ิตี้ิตี้รักลูกรักลูก


In [67]:
# ตรวจสอบความยาวของ 'instruction', 'input', และ 'output' ใน train และ val dataset
max_input_length = max(len(example['instruction'] + " " + example['input']) for example in train_dataset)
max_output_length = max(len(example['output']) for example in train_dataset)

print(f"Maximum input length: {max_input_length}")
print(f"Maximum output length: {max_output_length}")
print("Max sequence length:", model.config.max_position_embeddings)
print("Tokenizer pad token ID:", tokenizer.pad_token_id)
print("Tokenizer EOS token ID:", tokenizer.eos_token_id)

Maximum input length: 2477
Maximum output length: 2983
Max sequence length: 512
Tokenizer pad token ID: 1
Tokenizer EOS token ID: 6


In [92]:
# 🔹 5. Tokenization ข้อมูล
def check_labels_length(labels, max_length=512):
    # ตรวจสอบให้แน่ใจว่าแต่ละ label มีความยาวไม่เกิน max_length
    new_labels = []
    for label in labels:
        # แปลง label ให้เป็นลิสต์ถ้าเป็น int
        if isinstance(label, int):
            label = [label]
        if len(label) > max_length:
            print("Warning: label length exceeds max_length:", len(label))
        new_labels.append(label)
    return new_labels

def tokenize_function(examples):
    # ตรวจสอบ Input ว่ามีข้อมูลหรือไม่ ถ้าไม่มีให้ใช้ค่าว่าง
    if not examples["input"]:
        examples["input"] = [""] * len(examples["instruction"])

    # Combine instruction and input
    inputs = [instruction + " " + inp for instruction, inp in zip(examples["instruction"], examples["input"])]

    # Tokenize inputs with padding to max_length (512)
    model_inputs = tokenizer(
        inputs, 
        padding="max_length", 
        truncation=True, 
        max_length=512  # ปรับให้เหมาะสมกับความยาวของข้อความ
    )

    # Tokenize labels (output)
    labels = tokenizer(
        examples["output"], 
        padding="max_length",
        truncation=True,
        max_length=512  # ปรับให้เหมาะสมกับความยาวของข้อความ
    )

    # Handle labels with EOS token (ID 6)
    labels_ids = labels.get("input_ids", [])
    for i, label in enumerate(labels_ids):
        if isinstance(label, list) and tokenizer.eos_token_id in label:
            labels_ids[i] = label[:label.index(tokenizer.eos_token_id) + 1]

    
    # ตรวจสอบขนาดของ labels
    labels_ids = check_labels_length(labels_ids)

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels_ids
    return model_inputs

# ใช้ map เพื่อแปลงข้อมูลทั้งหมด
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 10352/10352 [00:03<00:00, 3417.93 examples/s]


In [98]:
# ทดสอบการโหลดข้อมูลและ tokenization กับข้อมูลบางส่วน
sample = train_dataset[0]  # เลือกตัวอย่างจาก train_dataset
print("Instruction:", sample["instruction"])
print("Input:", sample["input"])
print("Output:", sample["output"])

# ใช้ tokenize_function กับตัวอย่างนี้
tokenized_sample = tokenize_function(sample)
print("Tokenized Inputs:", tokenized_sample["input_ids"])
print("Tokenized attention_mask:", tokenized_sample["attention_mask"])
print("Tokenized Labels:", tokenized_sample["labels"])

Instruction:  ค้นหาข่าวสารล่าสุดเกี่ยวกับการเลือกตั้งประธานาธิบดีสหรัฐฯ
Input: 
Output:  ต่อไปนี้เป็นพาดหัวข่าวยอดนิยมจากวันที่ 2 พฤศจิกายน 2021: "ศาลสูงสหรัฐปฏิเสธการเสนอราคาของทรัมป์เพื่อบล็อกการเผยแพร่บันทึกเมื่อวันที่ 6 มกราคม - npr.org" โปรดทราบว่าฉันเป็น AI ที่ได้รับการฝึกฝนเกี่ยวกับข้อมูลจนถึงกลางเดือนตุลาคม 2021 ดังนั้นหากมีการอัปเดตล่าสุดที่คุณต้องการ คุณอาจต้องการใช้เครื่องมือค้นหาหรือแหล่งข่าวสำหรับบทความล่าสุด
Tokenized Inputs: [[5, 10, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
# 🔹 6. ทดลอง train model

In [12]:
# 🔹 7. Training Arguments
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,              
    per_device_train_batch_size=2,   
    per_device_eval_batch_size=2,    
    gradient_accumulation_steps=8,   
    num_train_epochs=2,              
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,             
    report_to="none",
    fp16=True,                      
    save_total_limit=1,
    load_best_model_at_end=True
)

c:\Users\MSI T\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# 🔹 8. สร้าง Trainer
trainer = Trainer(
    model=model,                     # โมเดลที่ใช้ฝึก
    args=training_args,              # argument ที่กำหนด
    train_dataset=train_dataset,     # ข้อมูลฝึก
    eval_dataset=val_dataset,        # ข้อมูลทดสอบ
    tokenizer=tokenizer,             # tokenizer
    data_collator=data_collator      # data collator
)

C:\Users\MSI T\AppData\Local\Temp\ipykernel_43956\1791213984.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# 🔹 9. ฝึกสอนโมเดล
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 🔹 10. บันทึกโมเดลที่ฝึกเสร็จ
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_tokenizer")

In [ ]:
# 🔹 11. ทดสอบโมเดลหลังจากฝึก
def chat_with_ai(prompt):
    inputs = tokenizer(f"<s>[INST] {prompt} [/INST]", return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=200).to("cuda")
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ทดสอบโมเดล
print(chat_with_ai("อธิบายหลักการทำงานของปัญญาประดิษฐ์"))